# **Imports**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import graphviz
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import export_graphviz

# **Data Cleaning**

In [ ]:
# read in csv as dataframe
df_salaries_uncleaned = pd.read_csv("data/Salary_Data.csv")

# drop null entries
df_salaries_cleaned = df_salaries_uncleaned.dropna()

# standardize the degree names
df_salaries_cleaned["Education Level"] = df_salaries_cleaned["Education Level"].replace("Bachelor's Degree", "Bachelor's")
df_salaries_cleaned["Education Level"] = df_salaries_cleaned["Education Level"].replace("Master's Degree", "Master's")
df_salaries_cleaned["Education Level"] = df_salaries_cleaned["Education Level"].replace("phD", "PhD")

# normalize data types
df_salaries_cleaned["Age"] = df_salaries_cleaned["Age"].astype("int")
df_salaries_cleaned["Years of Experience"] = df_salaries_cleaned["Years of Experience"].astype("int")

# drop duplicate entries
df_salaries_cleaned = df_salaries_cleaned.drop_duplicates()

# show data summary
df_salaries_cleaned.info()
df_salaries_cleaned.head()

# **Data Exploration**

In [ ]:
# Figue 1: Overall Data Distribution with Histograms
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
df_salaries_cleaned["Age"].plot(kind="hist", ax=axes[0, 0], title="Age")
df_salaries_cleaned["Years of Experience"].plot(kind="hist", ax=axes[0, 1], title="Years of Experience")
df_salaries_cleaned["Salary"].plot(kind="hist", ax=axes[0, 2], title="Salary")
df_salaries_cleaned["Gender"].value_counts().plot(kind="bar", ax=axes[1, 0], title="Gender")
df_salaries_cleaned["Education Level"].value_counts().plot(kind="bar", ax=axes[1, 1], title="Education")
df_salaries_cleaned["Job Title"].value_counts()[:20].plot(kind="bar", ax=axes[1, 2], title="Top 20 Job Titles")
fig.suptitle("Overall Data Distribution", fontsize=16)
plt.tight_layout()

# Figure 2: Data distribution - Salaries v. one factor
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 7))
xfactor = "Gender"
g = sns.boxplot(x=xfactor, y="Salary", data=df_salaries_cleaned, ax=axes[0, 0], order=df_salaries_cleaned.groupby(xfactor)["Salary"].median().sort_values().index)
g.set(title='Gender', xlabel=None)

xfactor = "Education Level"
g = sns.boxplot(x=xfactor, y="Salary", data=df_salaries_cleaned, ax=axes[0, 1], order=df_salaries_cleaned.groupby(xfactor)["Salary"].median().sort_values().index)
g.set(title="Education Level", xlabel=None)

xfactor = "Age"
plt.xticks(rotation=90)
g = sns.boxplot(x=xfactor, y="Salary", ax=axes[1, 0], data=df_salaries_cleaned)
g.set(title="Age", xlabel=None)

xfactor = "Years of Experience"
plt.xticks(rotation=90)
g = sns.boxplot(x=xfactor, y="Salary", ax=axes[1, 1], data=df_salaries_cleaned)
g.set(title="Years of Experience", xlabel=None)

fig.suptitle("Data distribution - Salaries v. one factor", fontsize=16)
plt.tight_layout()

# Figure 3 & 4: Salary Distributions by Education Level and Years of Experience
g = sns.displot(data=df_salaries_cleaned, x="Salary", hue="Education Level", multiple="stack")
g.set(title='Salary Distribution By Education Level', xlabel=None)
plt.xticks(rotation=45)
g = sns.displot(data=df_salaries_cleaned, x="Salary", hue="Years of Experience", multiple="stack")
g.set(title="Salary Distribution By Years of Experience", xlabel=None)
p = plt.xticks(rotation=45)

In [ ]:
# test multiple different feature combinations
df_salaries_cleaned_edulvl = df_salaries_cleaned
df_salaries_cleaned_edulvl["Education Level"] = df_salaries_cleaned["Education Level"].replace("High School", 1)
df_salaries_cleaned_edulvl["Education Level"] = df_salaries_cleaned["Education Level"].replace("Bachelor's", 2)
df_salaries_cleaned_edulvl["Education Level"] = df_salaries_cleaned["Education Level"].replace("Master's", 3)
df_salaries_cleaned_edulvl["Education Level"] = df_salaries_cleaned["Education Level"].replace("PhD", 4)

# Multivariate Salary Model with Education Level and Years of Experience
X = df_salaries_cleaned_edulvl[["Education Level", "Years of Experience"]].values.reshape(-1,2)
Y = df_salaries_cleaned_edulvl["Salary"]

x = X[:, 0]
y = X[:, 1]
z = Y
x_pred = np.linspace(0, 4)
y_pred = np.linspace(0, 35)
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

ols = LinearRegression()
model = ols.fit(X, Y)
predicted = model.predict(model_viz)

r2 = model.score(X, Y)
plt.style.use("default")

fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131, projection="3d")
ax2 = fig.add_subplot(132, projection="3d")
ax3 = fig.add_subplot(133, projection="3d")

axes = [ax1, ax2, ax3]

for ax in axes:
    ax.plot(x, y, z, color="k", zorder=15, linestyle="none", marker="o", alpha=0.01)
    ax.scatter(xx_pred.flatten(), yy_pred.flatten(), predicted, facecolor=(0,0,0,0), s=5, edgecolor="#70b3f0")
    ax.set_xlabel("Education Level", fontsize=12)
    ax.set_ylabel("Years of Experience", fontsize=12)
    ax.set_zlabel("Salary", fontsize=12)

ax1.view_init(elev=28, azim=120)
ax2.view_init(elev=4, azim=114)
ax3.view_init(elev=60, azim=165)

fig.suptitle("$R^2 = %.2f$" % r2, fontsize=20)

fig.tight_layout()

# Multivariate Salary Model with Age and Years of Experience
X = df_salaries_cleaned[["Age", "Years of Experience"]].values.reshape(-1,2)
Y = df_salaries_cleaned["Salary"]

x = X[:, 0]
y = X[:, 1]
z = Y

x_pred = np.linspace(0,62)
y_pred = np.linspace(0, 35)
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

ols = LinearRegression()
model = ols.fit(X, Y)
predicted = model.predict(model_viz)

r2 = model.score(X, Y)
plt.style.use("default")

fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131, projection="3d")
ax2 = fig.add_subplot(132, projection="3d")
ax3 = fig.add_subplot(133, projection="3d")

axes = [ax1, ax2, ax3]

for ax in axes:
    ax.plot(x, y, z, color="k", zorder=15, linestyle="none", marker="o", alpha=0.01)
    ax.scatter(xx_pred.flatten(), yy_pred.flatten(), predicted, facecolor=(0,0,0,0), s=5, edgecolor='#70b3f0')
    ax.set_xlabel("Age", fontsize=12)
    ax.set_ylabel("Years of Experience", fontsize=12)
    ax.set_zlabel("Salary", fontsize=12)

ax1.view_init(elev=28, azim=120)
ax2.view_init(elev=4, azim=114)
ax3.view_init(elev=60, azim=165)

fig.suptitle("$R^2 = %.2f$" % r2, fontsize=20)

fig.tight_layout()

# **Data Preprocessing**

In [ ]:
# separate categorical and numerical columns
categorical_columns = ["Gender", "Education Level", "Job Title"]
numerical_columns = ["Age", "Years of Experience"]

# separate the data into features and target values
X_data = df_salaries_cleaned[["Age", "Gender", "Education Level", "Job Title", "Years of Experience"]]
y_data = df_salaries_cleaned["Salary"]

# split the data into test and training data
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# transform table to standardize columns
ct = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), categorical_columns),
    (StandardScaler(), numerical_columns),
    remainder="drop"
)

# **Linear Regression**

### Hyperparameter Tuning

In [ ]:
# create tuning pipeline
pipeline_lin_initial = make_pipeline(
    ct,
    SelectKBest(score_func=f_regression),
    LinearRegression()
)

# hyperparameters to tune
param_grid_lin = {
    "selectkbest__k": [1, 2, 3, 4, 5],
}

# create an exhaustive search using cross validation
grid_search_lin = GridSearchCV(pipeline_lin_initial, param_grid_lin, cv=5, scoring="r2")

# perform exhaustive search to tune hyperparameters
grid_search_lin.fit(X_data, y_data)

print("Best Features:", grid_search_lin.best_estimator_.feature_names_in_)

### Model Training

In [ ]:
# create model pipeline
pipeline_lin_tuned = make_pipeline(
    ct,
    LinearRegression()
)

# train model on training data
pipeline_lin_tuned.fit(X_train, y_train)

### Model Evaluations

In [ ]:
# predict salaries on testing data
y_pred_linear = pipeline_lin_tuned.predict(X_test)

print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred_linear))
print("Root Mean Squared Error (RMSE):", math.sqrt(mean_squared_error(y_test, y_pred_linear)))
print("R2 Score:", r2_score(y_test, y_pred_linear))

### Model Visualizations

In [ ]:
# Actual Salaries vs Predicted Salaries Graph

# table consisting of actual salaries
df_actual = pd.DataFrame({"x": [n for n in range(len(y_test))]})
df_actual["y"] = list(y_test)

# table consisting of predicted salaries
df_predicted = pd.DataFrame({"x": [n for n in range(len(y_pred_linear))]})
df_predicted["y"] = y_pred_linear

# create shared plot
ax = plt.gca()

# plot actual and predicted salaries
df_actual.plot(x="x", y="y", kind="line", color="blue", label="Actual", ax=ax, alpha=0.5)
df_predicted.plot(x="x", y="y", kind="line", color="green", label="Predicted", ax=ax, alpha=0.5)

# set labels
ax.set_xlabel("Index")
ax.set_ylabel("Salary")
ax.set_title("Actual Salaries vs Predicted Salaries for Linear Regression")

plt.show()

In [ ]:
# Absolute Error Graph

# take the difference between each actual and predicted
df_diff = pd.DataFrame({"x": [n for n in range(len(y_test))]})
df_diff["y"] = y_test.to_numpy() - y_pred_linear

# apply absolute value
df_diff = df_diff.apply(abs)

# set labels
plt.plot(df_diff["x"], df_diff["y"], color="purple", alpha=0.5)
plt.xlabel("Index")
plt.ylabel("Salary")
plt.title("Absolute Error for Linear Regression")

# **K-Nearest Neighbors**

### Hyperparameter Tuning

In [ ]:
# create tuning pipeline
pipeline_knn_initial = make_pipeline(
    ct,
    SelectKBest(score_func=f_regression),
    KNeighborsRegressor(metric="euclidean")
)

# hyperparameters to tune
param_grid_knn = {
    "selectkbest__k": [1, 2, 3, 4, 5],
    "kneighborsregressor__n_neighbors": [n for n in range(1, 10)]
}

# create an exhaustive search using cross validation
grid_search_knn = GridSearchCV(pipeline_knn_initial, param_grid_knn, cv=5, scoring="r2")

# perform exhaustive search to tune hyperparameters
grid_search_knn.fit(X_data, y_data)

print("Best N-Neighbors:", grid_search_knn.best_params_["kneighborsregressor__n_neighbors"])
print("Best Features:", grid_search_knn.best_estimator_.feature_names_in_)

### Model Training

In [ ]:
# create model pipeline
pipeline_knn_tuned = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=9, metric="euclidean")
)

# train model on training data
pipeline_knn_tuned.fit(X_train, y_train)

### Model Evaluations

In [ ]:
# predict salaries on testing data
y_pred_knn = pipeline_knn_tuned.predict(X_test)

print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred_knn))
print("Root Mean Squared Error (RMSE):", math.sqrt(mean_squared_error(y_test, y_pred_knn)))
print("R2 Score:", r2_score(y_test, y_pred_knn))

### Model Visualizations

In [ ]:
# Actual Salaries vs Predicted Salaries Graph

# table consisting of actual salaries
df_actual = pd.DataFrame({"x": [n for n in range(len(y_test))]})
df_actual["y"] = list(y_test)

# table consisting of predicted salaries
df_predicted = pd.DataFrame({"x": [n for n in range(len(y_pred_knn))]})
df_predicted["y"] = y_pred_knn

# create shared plot
ax = plt.gca()

# plot actual and predicted salaries
df_actual.plot(x="x", y="y", kind="line", color="blue", label="Actual", ax=ax, alpha=0.5)
df_predicted.plot(x="x", y="y", kind="line", color="green", label="Predicted", ax=ax, alpha=0.5)

# set labels
ax.set_xlabel("Index")
ax.set_ylabel("Salary")
ax.set_title("Actual Salaries vs Predicted Salaries for KNN")

plt.show()

In [ ]:
# Absolute Error Graph

# take the difference between each actual and predicted
df_diff = pd.DataFrame({"x": [n for n in range(len(y_test))]})
df_diff["y"] = y_test.to_numpy() - y_pred_knn

# apply absolute value
df_diff = df_diff.apply(abs)

# set labels
plt.plot(df_diff["x"], df_diff["y"], color="purple", alpha=0.5)
plt.xlabel("Index")
plt.ylabel("Salary")
plt.title("Absolute Error for KNN")

# **Random Forest**

### Hyperparameter Tuning

In [ ]:
# create model pipeline
pipeline_rf_initial = make_pipeline(
    ct,
    RandomForestRegressor(n_estimators=100, random_state=42)
)

# Define the hyperparameters grid for the Random Forest
param_grid_rf = {
    "randomforestregressor__n_estimators": [100, 200],  # Number of trees in the forest
    "randomforestregressor__max_depth": [None, 10, 20, 30],  # Maximum depth of the trees
    "randomforestregressor__min_samples_split": [2, 5, 10],  # Minimum number of samples required to split an internal node
    "randomforestregressor__min_samples_leaf": [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

# Initialize GridSearchCV with the pipeline and parameter grid
grid_search_rf = GridSearchCV(pipeline_rf_initial, param_grid_rf, cv=5, scoring="r2", verbose=1)

# Assuming X_data and y_data are your features and target variable
grid_search_rf.fit(X_data, y_data)

# Get the best hyperparameters
print("Best Hyperparameters:", grid_search_rf.best_params_)
print("Best Features:", grid_search_rf.best_estimator_.feature_names_in_)

### Model Training

In [ ]:
# create model pipeline
pipeline_rf_tuned = make_pipeline(
    ct,
    RandomForestRegressor(max_depth=30, min_samples_leaf=1, min_samples_split=5,n_estimators=200, random_state=42)
)

# train model on training data
pipeline_rf_tuned.fit(X=X_train, y=y_train)

### Model Evaluation

In [ ]:
# predict salaries on testing data
y_pred_rf = pipeline_rf_tuned.predict(X_test)

print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error (RMSE):", math.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R2 Score:", r2_score(y_test, y_pred_rf))

### Model Visualizations

In [ ]:
# function for extracting the features used from a pipeline's column transformer
def get_feature_names(column_transformer):
    """Get feature names from a ColumnTransformer."""
    output_features = []

    # loop through each transformer within the ColumnTransformer
    for name, estimator, columns in column_transformer.transformers_:
        if name == "remainder":  # Skip the 'remainder' transformer
            continue
        # handle one-hot encoded features
        if hasattr(estimator, "categories_"):
            for i, category in enumerate(estimator.categories_):
                output_features.extend([f"{columns[i]}_{cat}" for cat in category])
        # handle other transformers (e.g., scaling numerical features)
        else:
            output_features.extend(columns)

    return output_features

In [ ]:
random_forest_model = pipeline_rf_tuned.named_steps["randomforestregressor"]
tree = random_forest_model.estimators_[0]

# extract feature names
transformed_feature_names = get_feature_names(pipeline_rf_tuned.named_steps["columntransformer"])

# export the tree to a DOT format
dot_data = export_graphviz(tree, out_file=None, 
                           feature_names=transformed_feature_names, 
                           filled=True, rounded=True, 
                           special_characters=True, max_depth=3)

# render the DOT data
graph = graphviz.Source(dot_data)
graph.render("decision_tree")

In [ ]:
# Actual Salaries vs Predicted Salaries Graph

# table consisting of actual salaries
df_actual = pd.DataFrame({"x": [n for n in range(len(y_test))]})
df_actual["y"] = list(y_test)

# table consisting of predicted salaries
df_predicted = pd.DataFrame({"x": [n for n in range(len(y_pred_rf))]})
df_predicted["y"] = y_pred_rf

# create shared plot
ax = plt.gca()

# plot actual and predicted salaries
df_actual.plot(x="x", y="y", kind="line", color="blue", label="Actual", ax=ax, alpha=0.65)
df_predicted.plot(x="x", y="y", kind="line", color="green", label="Predicted", ax=ax, alpha=0.65)

# set labels
ax.set_xlabel("Index")
ax.set_ylabel("Salary")
ax.set_title("Actual Salaries vs Predicted Salaries for Random Forest Model")

plt.show()

In [ ]:
# Absolute Error Graph

# take the difference between each actual and predicted
df_diff = pd.DataFrame({"x": [n for n in range(len(y_test))]})
df_diff["y"] = y_test.to_numpy() - y_pred_rf

# apply absolute value
df_diff = df_diff.apply(abs)

# set labels
plt.plot(df_diff["x"], df_diff["y"], color="purple", alpha=0.65)
plt.xlabel("Index")
plt.ylabel("Salary")
plt.title("Absolute Error for Random Forest Model")

# **Example Prediction**

In [ ]:
X_test = pd.DataFrame({
    "Age": [50],
    "Gender": ["Male"],
    "Education Level": ["PhD"],
    "Job Title": ["Software Developer"],
    "Years of Experience": [5]
})

y_pred = pipeline_rf_tuned.predict(X_test)

print("Predicted Salary:", y_pred[0])